In [1]:
#Importing the libraries
import pandas as pd
import numpy as np

In [2]:
# Providing the absolute path to the Excel file
input_file_path = '/Users/namanparuthi/Desktop/Amex/round 2/Amex Campus Challenge 2024 input.xlsx'
sheet_name = 'batsman_scorecard'

In [3]:
df = pd.read_excel(input_file_path, sheet_name=sheet_name)

In [4]:
#Extracting the wicket-keepers from batsmen dataset
wk = df[df['is_batsman_keeper'] == 1]
elligible_wk= wk.groupby('batsman_id').filter(lambda x: len(x) >= 2)

In [5]:
# Calculating total runs, total balls, number of innings, 100s, and 50s for each wk
batsman_stats = elligible_wk.groupby('batsman_id').agg(
    total_runs=pd.NamedAgg(column='runs', aggfunc='sum'),
    innings=pd.NamedAgg(column='batsman_id', aggfunc='count'),
    hundreds=pd.NamedAgg(column='runs', aggfunc=lambda x: (x >= 100).sum()),
    fifties=pd.NamedAgg(column='runs', aggfunc=lambda x: ((x >= 50) & (x < 100)).sum()),
    strike_rate=pd.NamedAgg(column='strike_rate', aggfunc='mean')
).reset_index()

In [6]:
# Calculating average
batsman_stats['average'] = batsman_stats['total_runs'] / batsman_stats['innings']


In [7]:
#Defining functions to assing points
def assign_strike_rate_points(sr):
    if sr >= 150:
        return 50
    elif sr >= 100:
        return 40
    elif sr >= 80:
        return 30
    else:
        return 0

def assign_average_points(avg):
    if avg >= 50:
        return 30
    elif avg >= 40:
        return 20
    elif avg >= 30:
        return 10
    else:
        return 5

def assign_hundreds_points(hundreds):
    if hundreds >= 3:
        return 30
    elif hundreds == 2:
        return 20
    elif hundreds == 1:
        return 10
    else:
        return 0

def assign_fifties_points(fifties):
    if fifties >= 5:
        return 20
    elif fifties >= 3:
        return 10
    elif fifties >= 1:
        return 5
    else:
        return 0

# Applying the point assigning functions
batsman_stats['strike_rate_points'] = batsman_stats['strike_rate'].apply(assign_strike_rate_points)
batsman_stats['average_points'] = batsman_stats['average'].apply(assign_average_points)
batsman_stats['hundreds_points'] = batsman_stats['hundreds'].apply(assign_hundreds_points)
batsman_stats['fifties_points'] = batsman_stats['fifties'].apply(assign_fifties_points)

# Calculating total points
batsman_stats['total_points'] = (batsman_stats['strike_rate_points'] + 
                                    batsman_stats['average_points'] + 
                                    batsman_stats['hundreds_points'] + 
                                    batsman_stats['fifties_points'])

In [8]:
# Sorting total points in descending order
top_batsmen = batsman_stats.sort_values(by='total_points', ascending=False)

# Extracting the top ten wk
top_ten_batsmen = top_batsmen.head(10)

top_ten_batsmen

,batsman_id,total_runs,innings,hundreds,fifties,strike_rate,average,strike_rate_points,average_points,hundreds_points,fifties_points,total_points
31,2263736,3718,84,2,35,116.307143,44.261905,40,20,20,20,100
61,3057312,875,28,2,5,149.640357,31.250000,40,10,20,20,90
43,2654014,2650,90,2,20,119.989778,29.444444,40,5,20,20,85
107,4635658,1336,46,2,8,126.430000,29.043478,40,5,20,20,85
14,1663885,648,19,1,6,112.638947,34.105263,40,10,10,20,80
22,2082044,635,15,0,7,144.444000,42.333333,40,20,0,20,80
29,2162782,2082,61,1,18,125.848197,34.131148,40,10,10,20,80
57,2954769,535,12,0,5,121.716667,44.583333,40,20,0,20,80
126,6139370,1089,47,1,6,105.163191,23.170213,40,5,10,20,75
91,4003390,887,33,1,7,103.228182,26.878788,40,5,10,20,75


as there is only a single wicket keeper with the top points, recency and consistency do not come into the picture